# **Taller 2 - Simulación Computacional 2020-I**

Fecha de entrega: 21 de Septiembre de 2020



## 1. 
Implementar en Notebook Jupyter una red de colas de un Centro de Diagnóstico Automotriz que emiten la revisión técnico-mecánica [<a href="https://drive.google.com/file/d/0B0MRo23s_UhySlJVYU41QlBkaWs/view?usp=sharing">Sec. 5.5.2 Rios08 </a> ]. 

In [90]:
from datetime import datetime
import math 

class RedColas():
  def metodo_congruencial_mixto(cantidad=1, x0=None, a=None, c=None, m=None):
    pseudoaletorios=[]
    if not x0:
      x0=datetime.now().microsecond
    if not a:
      a=16807
    if not c:
      c=75873
    if not m:
      m=2**31-1  
    actual=x0
    for i in range(0,cantidad):
      xn=(a*actual+c)%m
      un=xn/m
      pseudoaletorios.append(un)
      actual=xn
    return pseudoaletorios
  def exponencial_aleatoria(landa):
    U=metodo_congruencial_mixto()[0]
    x=-(math.log(1-U)/float(landa))
    return x
  def normal_aleatoria(miu, sigma):
    U=metodo_congruencial_mixto()[0]
    Z=0.5*(1+math.erf((U-miu)/(sigma*math.sqrt(2))))
    x=sigma*Z+miu
    if x<0:
      x=0
    return x
  # Variables Auxiliares
  M = 99999
  T = 100
  landa = 10
  cte = {"λ":10, "μ1":15, "σ1":5, "μ2":10, "μ31":20, "σ31":25, "μ32":10, "σ32":15}
  # Variables colas
  t=tsuc=Tp=0
  TSuc = {"tLL1" : M, "tS1" : M, "tS2" : M, "tS3" : M}
  NLL1=NLL2=NLL3=NS1=NS2=NS3=n1=n2=n3=0
  LL1=LL2=LL3=S1=S2=S3=[0]
  n_med_n1=n_med_n2=n_med_n3=0
  def iniciar(self): 
    X=exponencial_aleatoria(self.landa)
    if X>self.T:
      Tp = t_med_sistema = 0
      self.n_med_n1=self.n_med_n2=self.n_med_n3=0
    else:
      self.llegada_cliente(X)
      while (self.TSuc["tLL1"]!=self.M) or (self.TSuc["tS1"]!=self.M) or (self.TSuc["tS2"]!=self.M) or (self.TSuc["tS3"]!=self.M):  
        if min(self.TSuc.values())==self.TSuc["tLL1"]:
          tsuc=self.TSuc["tLL1"]
          self.TSuc["tLL1"]=self.M
          self.llegada_cliente(tsuc)     
        if min(self.TSuc.values())==self.TSuc["tS1"]:
          tsuc=self.TSuc["tS1"]
          self.TSuc["tS1"]=self.M
          self.servicio_nodo1(tsuc)
        if min(self.TSuc.values())==self.TSuc["tS2"]:
          tsuc=self.TSuc["tS2"]
          self.TSuc["tS2"]=self.M
          self.servicio_nodo2(tsuc)
        if min(self.TSuc.values())==self.TSuc["tS3"]:
          tsuc=self.TSuc["tS3"]
          self.TSuc["tS3"]=self.M
          self.servicio_nodo3(tsuc)     
      Tp=max(0, (self.t-self.T))
      acumulo1=acumulo2=acumulo3=0
      for ind in range(self.NLL1):
        acumulo1+=self.S1[ind]-self.LL1[ind]
      for ind in range(self.NLL2):
        acumulo2+=self.S2[ind]-self.LL2[ind]
      for ind in range(self.NLL3):
        acumulo3+=self.S3[ind]-self.LL3[ind]
      t_med_sistema=(acumulo1/self.NLL1)+(0.4*acumulo2/self.NLL2)+(acumulo3/self.NLL3)
      self.n_med_n1=self.n_med_n1/self.t
      self.n_med_n2=self.n_med_n2/self.t
      self.n_med_n3=self.n_med_n3/self.t
      print("Número medio de clientes en el servidor 1: {}".format(self.n_med_n1))
      print("Número medio de clientes en el servidor 2: {}".format(self.n_med_n2))
      print("Número medio de clientes en el servidor 3: {}".format(self.n_med_n3))
      print("Tiempo medio que pasan los clientes en el sistema: {}".format(t_med_sistema))
      print("Tiempo transcurrido desde T hasta que el último cliente abandona el sistema: {}".format(Tp))
  def llegada_cliente(self, tsuc):
    self.n_med_n1+=self.n1*(tsuc-self.t)
    self.n1+=1
    self.NLL1+=1
    self.LL1.insert(self.NLL1, tsuc)
    self.t=tsuc
    Y=exponencial_aleatoria(self.cte["λ"])
    if (self.t+Y)<self.T:
      self.TSuc["tLL1"]=(self.t + Y)
    if self.n1==1:
      Y=normal_aleatoria(self.cte["μ1"], self.cte["σ1"])
      self.TSuc["tS1"]=(self.t  + Y)
          
  def servicio_nodo1(self, tsuc):
    self.n_med_n1+=self.n1*(tsuc-self.t)
    self.n1-=1
    self.NS1+=1
    self.S1.insert(self.NS1, tsuc)
    U=metodo_congruencial_mixto()[0]
    if U<=0.4:
        self.n_med_n2+=self.n2*(tsuc-self.t)
        self.n2+=1
        self.NLL2+=1
        self.LL2.insert(self.NLL2, tsuc)
        if self.n2==1:
            Z=exponencial_aleatoria(self.cte["μ2"])
            self.TSuc["tS2"]=tsuc+Z
    else:
        self.n_med_n3+=self.n3*(tsuc-self.t)
        self.n3+=1
        self.NLL3+=1
        self.LL3.insert(self.NLL3, tsuc)
        if self.n3==1:
            W=normal_aleatoria(self.cte["μ31"], self.cte["σ31"])
            self.TSuc["tS3"]=tsuc+W
    self.t=tsuc
    if self.n1>0:
        S=normal_aleatoria(self.cte["μ1"], self.cte["σ1"])
        self.TSuc["tS1"]=self.t + S
  def servicio_nodo2(self, tsuc):
    self.n_med_n2+=self.n2*(tsuc - self.t)
    self.n2-=1
    self.NS2+=1
    self.S2.insert(self.NS2, tsuc)
    if self.n2>0:
      Y=exponencial_aleatoria(self.cte["μ2"])
      self.TSuc["tS2"]=tsuc+Y
    self.n_med_n3+=self.n3*(tsuc-self.t)
    self.n3+=1
    self.NLL3+=1
    self.LL3.insert(self.NLL3, tsuc)
    if self.n3==1:
      W=normal_aleatoria(self.cte["μ31"], self.cte["σ31"])
      self.TSuc["tS3"]=tsuc+W
    self.t=tsuc
  def servicio_nodo3(self, tsuc):
      self.n_med_n3+=self.n3*(tsuc-self.t)
      self.n3-=1
      self.NS3+=1
      self.S3.insert(self.NS3, tsuc)
      if self.n3>0:
          if self.n3<5:
              R=normal_aleatoria(self.cte["μ31"], self.cte["σ31"])
          else:
              R=normal_aleatoria(self.cte["μ32"], self.cte["σ32"])
          self.TSuc["tS3"]=tsuc+R
      self.t=tsuc
rc_obj = RedColas()
rc_obj.iniciar()

Número medio de clientes en el servidor 1: 847.3326205430326
Número medio de clientes en el servidor 2: 0.0003965046514821826
Número medio de clientes en el servidor 3: 4.452433861729388
Tiempo medio que pasan los clientes en el sistema: 0.0
Tiempo transcurrido desde T hasta que el último cliente abandona el sistema: 42562.043842764586


2.Implementar en Notebook Jupyter los siguientes ejemplos de SimPy:

a) <a href="https://simpy.readthedocs.io/en/latest/examples/latency.html">Event Latency</a>

b) <a href="https://simpy.readthedocs.io/en/latest/examples/process_communication.html">Process Communication</a>

c) <a href="https://simpy.readthedocs.io/en/latest/examples/carwash.html">Carwash</a>

d) <a href="https://simpy.readthedocs.io/en/latest/examples/gas_station_refuel.html">Gas Station Refueling</a>

e) <a href="https://simpy.readthedocs.io/en/latest/examples/movie_renege.html">Movie Renege</a>

f) <a href="https://simpy.readthedocs.io/en/latest/examples/machine_shop.html">Machine Shop</a>

a)

In [92]:
import simpy


SIM_DURATION = 100


class Cable(object):
    """This class represents the propagation through a cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('Sender sent this at %d' % env.now)


def receiver(env, cable):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Received this at %d while %s' % (env.now, msg))


# Setup and start the simulation
print('Event Latency')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Event Latency
Received this at 15 while Sender sent this at 5
Received this at 20 while Sender sent this at 10
Received this at 25 while Sender sent this at 15
Received this at 30 while Sender sent this at 20
Received this at 35 while Sender sent this at 25
Received this at 40 while Sender sent this at 30
Received this at 45 while Sender sent this at 35
Received this at 50 while Sender sent this at 40
Received this at 55 while Sender sent this at 45
Received this at 60 while Sender sent this at 50
Received this at 65 while Sender sent this at 55
Received this at 70 while Sender sent this at 60
Received this at 75 while Sender sent this at 65
Received this at 80 while Sender sent this at 70
Received this at 85 while Sender sent this at 75
Received this at 90 while Sender sent this at 80
Received this at 95 while Sender sent this at 85


b)

In [93]:
import random

import simpy


RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """A Broadcast pipe that allows one process to send messages to many.

    This construct is useful when message consumers are running at
    different rates than message generators and provides an event
    buffering to the consuming processes.

    The parameters are used to create a new
    :class:`~simpy.resources.store.Store` instance each time
    :meth:`get_output_conn()` is called.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Get a new output connection for this broadcast pipe.

        The return value is a :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s says hello at %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('LATE Getting Message: at time %d: %s received message: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('at time %d: %s received message: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('Process communication')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generator A', env, pipe))
env.process(message_consumer('Consumer A', env, pipe))

print('\nOne-to-one pipe communication\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generator A', env, bc_pipe))
env.process(message_consumer('Consumer A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumer B', env, bc_pipe.get_output_conn()))

print('\nOne-to-many pipe communication\n')
env.run(until=SIM_TIME)

Process communication

One-to-one pipe communication

at time 6: Consumer A received message: Generator A says hello at 6.
at time 12: Consumer A received message: Generator A says hello at 12.
at time 19: Consumer A received message: Generator A says hello at 19.
at time 26: Consumer A received message: Generator A says hello at 26.
at time 36: Consumer A received message: Generator A says hello at 36.
at time 46: Consumer A received message: Generator A says hello at 46.
at time 52: Consumer A received message: Generator A says hello at 52.
at time 58: Consumer A received message: Generator A says hello at 58.
LATE Getting Message: at time 66: Consumer A received message: Generator A says hello at 65
at time 75: Consumer A received message: Generator A says hello at 75.
at time 85: Consumer A received message: Generator A says hello at 85.
at time 95: Consumer A received message: Generator A says hello at 95.

One-to-many pipe communication

at time 10: Consumer A received message: G

c)

In [94]:
import random

import simpy


RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes


class Carwash(object):
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME)
        print("Carwash removed %d%% of %s's dirt." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    print('%s arrives at the carwash at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s enters the carwash at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Create a carwash, a number of initial cars and keep creating cars
    approx. every ``t_inter`` minutes."""
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, 'Car %d' % i, carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Car %d' % i, carwash))


# Setup and start the simulation
print('Carwash')
print('Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

Carwash
Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 5.00.
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 5 arrives at the carwash at 10.00.
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Carwash removed 97% of Car 4's dirt.
Carwash removed 56% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leaves the carwash at 15.00.
Car 6 arrives at the carwash at 16.00.
Car 6 enters the carwash 

d)

In [95]:
import itertools
import random

import simpy


RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds


def car(name, env, gas_station, fuel_pump):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of gas from it. If the stations reservoir is
    depleted, the car has to wait for the tank truck to arrive.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s arriving at gas station at %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Request one of the gas pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s finished refueling in %.1f seconds.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Periodically check the level of the *fuel_pump* and call the tank
    truck if the level falls below a threshold."""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print('Calling tank truck at %d' % env.now)
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds


def tank_truck(env, fuel_pump):
    """Arrives at the gas station after a certain delay and refuels it."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Tank truck arriving at time %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Tank truck refuelling %.1f liters.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, gas_station, fuel_pump))


# Setup and start the simulation
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

Gas Station refuelling
Car 0 arriving at gas station at 87.0
Car 0 finished refueling in 18.5 seconds.
Car 1 arriving at gas station at 129.0
Car 1 finished refueling in 19.0 seconds.
Car 2 arriving at gas station at 284.0
Car 2 finished refueling in 21.0 seconds.
Car 3 arriving at gas station at 385.0
Car 3 finished refueling in 13.5 seconds.
Car 4 arriving at gas station at 459.0
Calling tank truck at 460
Car 4 finished refueling in 22.0 seconds.
Car 5 arriving at gas station at 705.0
Car 6 arriving at gas station at 750.0
Tank truck arriving at time 760
Tank truck refuelling 188.0 liters.
Car 6 finished refueling in 29.0 seconds.
Car 5 finished refueling in 76.5 seconds.
Car 7 arriving at gas station at 891.0
Car 7 finished refueling in 13.0 seconds.


e)

In [96]:
import collections
import random

import simpy


RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SIM_TIME = 120  # Simulate until


def moviegoer(env, movie, num_tickets, theater):
    """A moviegoer tries to by a number of tickets (*num_tickets*) for
    a certain *movie* in a *theater*.

    If the movie becomes sold out, she leaves the theater. If she gets
    to the counter, she tries to buy a number of tickets. If not enough
    tickets are left, she argues with the teller and leaves.

    If at most one ticket is left after the moviegoer bought her
    tickets, the *sold out* event for this movie is triggered causing
    all remaining moviegoers to leave.

    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn or if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            return

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(0.5)
            return

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Setup and start the simulation
print('Movie renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('Movie "%s" sold out %.1f minutes after ticket counter '
              'opening.' % (movie, theater.when_sold_out[movie]))
        print('  Number of people leaving queue when film sold out: %s' %
              theater.num_renegers[movie])

Movie renege
Movie "Python Unchained" sold out 38.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 16
Movie "Kill Process" sold out 43.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 5
Movie "Pulp Implementation" sold out 28.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 5


f)

In [97]:
import random

import simpy


RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Return actual processing time for a concrete part."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Return time until next failure for a machine."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *repairman* and continues the production
    after the it is repaired.

    A machine has a *name* and a numberof *parts_made* thus far.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce parts as long as the simulation runs.

        While making a part, the machine may break multiple times.
        Request a repairman when this happens.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Break the machine every now and then."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """The repairman's other (unimportant) job."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Setup and start the simulation
print('Machine shop')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Machine %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Machine shop results after %s weeks' % WEEKS)
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))

Machine shop
Machine shop results after 4 weeks
Machine 0 made 3251 parts.
Machine 1 made 3273 parts.
Machine 2 made 3242 parts.
Machine 3 made 3343 parts.
Machine 4 made 3387 parts.
Machine 5 made 3244 parts.
Machine 6 made 3269 parts.
Machine 7 made 3185 parts.
Machine 8 made 3302 parts.
Machine 9 made 3279 parts.


3.Implementar una red de colas como la del punto 1 ([<a href="https://drive.google.com/file/d/0B0MRo23s_UhySlJVYU41QlBkaWs/view?usp=sharing">Sec. 5.5.2 Rios08 </a> ]) usando SimPy.